# Imports

In [2]:
## imports and environment variables

# Imports
import import_ipynb
import pandas as pd
import os
import re
import json
import data_repo_client
from google.cloud import bigquery
import ingest_pipeline_utilities as utils
import logging
from time import sleep

# Configure logging format
logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p", level=logging.INFO)
pd.set_option('display.max_rows', None)

# Environment variables
ws_bucket = os.environ["WORKSPACE_BUCKET"]


importing Jupyter notebook from ingest_pipeline_utilities.ipynb
Version 1.0.19: 2/23/2023 11:02am - Nate Calvanese - Made addition of auth domains as snapshot readers configurable
importing Jupyter notebook from source_files_creation.ipynb
Version 1.0.8: 10/12/2022 3:50pm - Nate Calvanese - Reverted source table prefixing
importing Jupyter notebook from build_file_inventory.ipynb
Version 1.0.4: 10/12/2022 10:26am - Nate Calvanese - Added support for specifying additional buckets/dirs
importing Jupyter notebook from process_table_data.ipynb
Version: 1.0.8: 1/31/2023 10:52am - Nate Calvanese - Added logic to deal with workspaces that are completely empty
importing Jupyter notebook from build_mapping_query.ipynb
Version 1.0.12: 2/21/2023 2:50pm - Nate Calvanese - Added support for $BQ_DATASET substitution variable
importing Jupyter notebook from output_data_validation.ipynb
Version 2.0.5: 10/25/2022 9:38am -- Removed auto-flagging of fileref columns with nulls
importing Jupyter notebook f

# Add and populate anvil_file.is_supplementary

## Script to patch dataset

In [4]:
def add_and_populate_supp_file_flg(dataset_id):
    logging.info(f"Processing anvil_file.is_supplementary for Dataset ID = {dataset_id}")
    
    # Retrieve dataset information
    logging.info("Retrieving necessary information from TDR.")
    src_schema_dict = {}
    api_client = utils.refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    try:
        response = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA", "ACCESS_INFORMATION"]).to_dict()
        src_schema_dict["tables"] = response["schema"]["tables"]
        bq_project = response["access_information"]["big_query"]["project_id"]
        bq_dataset = response["access_information"]["big_query"]["dataset_name"]
    except Exception as e:
        logging.error("Error retrieving information from TDR. Exiting function. Error: {}".format(e))
        return "Failure"
    
    # Determine if field needs to be added, and add if so
    logging.info("Adding anvil_file.is_supplementary to dataset schema, if necessary.")
    field_found = False
    for table in src_schema_dict["tables"]:
        if table["name"] == "anvil_file":
            for col in table["columns"]:
                if col["name"] == "is_supplementary":
                    field_found = True
                    logging.info("Field already found! Skipping schema update.")
                    break
            break
    if field_found == False:
        logging.info("Field not found. Running dataset schema update.")
        schema_update_request = {
            "description": "Adding is_supplementary column to anvil_file",
            "changes": {
                "addColumns": [
                  {
                    "tableName": "anvil_file",
                    "columns": [
                      {
                        "name": "is_supplementary",
                        "datatype": "boolean",
                        "array_of": False,
                        "required": False
                      }
                    ]
                  }
                ]
            }
        }
        attempt_counter = 0
        while True:
            try:
                schema_update_result, job_id = utils.wait_for_tdr_job(datasets_api.update_schema(id=dataset_id, dataset_schema_update_model=schema_update_request))
                logging.info("Dataset schema update succeeded!")
                break
            except Exception as e:
                logging.error("Error on dataset schema update: {}".format(str(e)))
                attempt_counter += 1
                if attempt_counter < 2:
                    logging.info("Retrying dataset schema update (attempt #{})...".format(str(attempt_counter)))
                    sleep(15)
                    continue
                else:
                    logging.error("Maximum number of retries exceeded. Unable to update dataset schema. Exiting function.")
                    return "Failure"
        
    # Re-process anvil_file data to include is_supplementary (where appropriate) and ingest into TDR dataset (as replace)
    logging.info("Re-processing existing anvil_file data to include is_supplementary value.")
    client = bigquery.Client()
    target_file = "anvil_file.json"
    destination_dir = "ingest_pipeline/output/transformed/anvil/{}/table_data".format(dataset_id)
    query = """WITH activity_agg
        AS
        (
          SELECT used_biosample_id, generated_file_id, used_file_id FROM `{project}.{dataset}.anvil_activity`
          UNION ALL 
          SELECT [] AS used_biosample_id, generated_file_id, used_file_id FROM `{project}.{dataset}.anvil_alignmentactivity`
          UNION ALL 
          SELECT used_biosample_id, generated_file_id, [] AS used_file_id FROM `{project}.{dataset}.anvil_assayactivity`
          UNION ALL 
          SELECT used_biosample_id, generated_file_id, [] AS used_file_id FROM `{project}.{dataset}.anvil_sequencingactivity`
          UNION ALL 
          SELECT [] AS used_biosample_id, generated_file_id, used_file_id FROM `{project}.{dataset}.anvil_variantcallingactivity`
        ),
        activity_exp
        AS
        (
          SELECT file_id, int_file_id, biosample_id
          FROM activity_agg
            LEFT JOIN UNNEST(used_biosample_id) AS biosample_id
            LEFT JOIN UNNEST(generated_file_id) as file_id
            LEFT JOIN UNNEST(used_file_id) as int_file_id
        ), 
        act_exp_lookup
        AS
        (
          SELECT file_id, MAX(biosample_id) AS biosample_id
          FROM
          (
            SELECT file_id, biosample_id
            FROM activity_exp
            WHERE int_file_id IS NULL AND file_id IS NOT NULl AND biosample_id IS NOT NULL
            UNION ALL
            SELECT a2.file_id, a1.biosample_id
            FROM activity_exp a1
              LEFT JOIN activity_exp a2
              ON a1.file_id = a2.int_file_id
            WHERE a2.int_file_id IS NOT NULL AND a2.file_id IS NOT NULL AND a1.biosample_id IS NOT NULL
          )
          GROUP BY file_id
        )
        SELECT t1.file_id, data_modality, file_format, file_size, file_md5sum, reference_assembly, file_name, file_ref, source_datarepo_row_ids,
        CASE WHEN t2.biosample_id IS NULL THEN TRUE ELSE FALSE END AS is_supplementary
        FROM `{project}.{dataset}.anvil_file` t1
          LEFT JOIN act_exp_lookup t2
          ON t1.file_id = t2.file_id
        """.format(project=bq_project, dataset=bq_dataset)
    try:
        df = client.query(query).result().to_dataframe()
        records_json = df.to_json(orient='records') 
        records_list = json.loads(records_json)
        records_cnt = len(records_list)
        with open(target_file, 'w') as outfile:
            for idx, val in enumerate(records_list):
                json.dump(val, outfile)
                if idx < (records_cnt - 1):
                    outfile.write('\n')
        !gsutil cp $target_file $ws_bucket/$destination_dir/ 2> stdout
        !rm $target_file
        logging.info("Successfully created new anvil_file.json file.")
    except Exception as e:
        logging.error("Error creating new anvil_file.json file. Exiting function. Error: {}".format(str(e)))
        return "Failure"

    # Ingest updated anvil_file data
    logging.info("Ingesting updated anvil_file data into TDR dataset.")
    source_full_file_path = "{}/{}/{}".format(ws_bucket, destination_dir, "anvil_file.json")
    ingest_request = {
        "table": "anvil_file",
        "profile_id": "e0e03e48-5b96-45ec-baa4-8cc1ebf74c61",
        "ignore_unknown_values": True,
        "resolve_existing_files": True,
        "updateStrategy": "replace",
        "format": "json",
        "load_tag": "Ingest for {}".format(dataset_id),
        "path": source_full_file_path
    }
    attempt_counter = 0
    while True:
        try:
            api_client = utils.refresh_tdr_api_client()
            datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
            ingest_request_result, job_id = utils.wait_for_tdr_job(datasets_api.ingest_dataset(id=dataset_id, ingest=ingest_request))
            logging.info("Ingest from file anvil_file.json succeeded: {}".format(str(ingest_request_result)[0:1000]))
            break
        except Exception as e:
            logging.error("Error on Dataset Ingest: {}".format(str(e)))
            attempt_counter += 1
            if attempt_counter < 2:
                logging.info("Retrying Dataset Ingest (attempt #{})...".format(str(attempt_counter)))
                sleep(10)
                continue
            else:
                logging.error("Maximum number of retries exceeded. Exiting function.")
                return "Failure"
    
    # Return success message if no failures recorded
    logging.info("Function completed successfully.")
    return "Success"

# Loop through datasets and process is_supplementary field
dataset_id_list = [
'd74b26d5-24bb-4696-84c3-bcd1f5f90b08',
]
results = []
for dataset_id in dataset_id_list:
    status = add_and_populate_supp_file_flg(dataset_id) 
    results.append([dataset_id, status])
    results_df = pd.DataFrame(results, columns = ["dataset_id", "run_status"])
display(results_df)
    

02/23/2023 05:30:02 PM - INFO: Processing anvil_file.is_supplementary for Dataset ID = d74b26d5-24bb-4696-84c3-bcd1f5f90b08
02/23/2023 05:30:02 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 05:30:02 PM - INFO: Adding anvil_file.is_supplementary to dataset schema, if necessary.
02/23/2023 05:30:02 PM - INFO: Field not found. Running dataset schema update.
TDR Job ID: duVM6UEaRsas-wxGHXj9tQ
02/23/2023 05:30:12 PM - INFO: Dataset schema update succeeded!
02/23/2023 05:30:12 PM - INFO: Re-processing existing anvil_file data to include is_supplementary value.
02/23/2023 05:30:22 PM - INFO: Successfully created new anvil_file.json file.
02/23/2023 05:30:22 PM - INFO: Ingesting updated anvil_file data into TDR dataset.
TDR Job ID: Lm19NntrT-a-3mZDy7aKlw
02/23/2023 05:30:52 PM - INFO: Ingest from file anvil_file.json succeeded: {'dataset_id': 'd74b26d5-24bb-4696-84c3-bcd1f5f90b08', 'dataset': 'ANVIL_CMG_UWash_GRU_20221020', 'table': 'anvil_file', 'path': 'gs://fc-2a9eefc3-03

,dataset_id,run_status
0,d74b26d5-24bb-4696-84c3-bcd1f5f90b08,Success


## Script to validate patch worked properly

In [ ]:
def validate_supp_file_flg(dataset_id):
    
    # Retrieve dataset information
    src_schema_dict = {}
    api_client = utils.refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    try:
        response = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA", "ACCESS_INFORMATION"]).to_dict()
        src_schema_dict["tables"] = response["schema"]["tables"]
        bq_project = response["access_information"]["big_query"]["project_id"]
        bq_dataset = response["access_information"]["big_query"]["dataset_name"]
    except Exception as e:
        return "Failure - Issue Retrieving Dataset Info"
    
    # Determine if field exists for dataset, continue if so, fail otherwise
    field_found = False
    for table in src_schema_dict["tables"]:
        if table["name"] == "anvil_file":
            for col in table["columns"]:
                if col["name"] == "is_supplementary":
                    field_found = True
                    break
            break
    if field_found == False:
        return "Failure - is_supplementary field not found"
    else:
        client = bigquery.Client()
        query = """SELECT COUNT(*) AS rec_cnt, COUNT(is_supplementary) AS populated_cnt
                    FROM `{project}.{dataset}.anvil_file`""".format(project=bq_project, dataset=bq_dataset)
        try:
            df = client.query(query).result().to_dataframe()
            if df["rec_cnt"].values[0] == df["populated_cnt"].values[0]:
                return "Success"
        except Exception as e:
            return "Failure - BigQuery Error"

# Loop through datasets and validate is_supplementary field
dataset_id_list = [
'd74b26d5-24bb-4696-84c3-bcd1f5f90b08',
]
results = []
for dataset_id in dataset_id_list:
    status = validate_supp_file_flg(dataset_id) 
    results.append([dataset_id, status])
    results_df = pd.DataFrame(results, columns = ["dataset_id", "validation_status"])
display(results_df)

# Attempt to populate anvil_donor.organism_type

## Script to patch dataset

In [ ]:
def populate_organism_type(dataset_id):
    logging.info(f"Processing anvil_donor.organism_type for Dataset ID = {dataset_id}")
    
    # Retrieve dataset information
    logging.info("Retrieving necessary information from TDR.")
    api_client = utils.refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    try:
        response = datasets_api.retrieve_dataset(id=dataset_id, include=["ACCESS_INFORMATION"]).to_dict()
        bq_project = response["access_information"]["big_query"]["project_id"]
        bq_dataset = response["access_information"]["big_query"]["dataset_name"]
    except Exception as e:
        logging.error("Error retrieving information from TDR. Exiting function. Error: {}".format(e))
        return "Failure"

    # Re-process anvil_donor data to include organism_type (where available)
    logging.info("Re-processing existing anvil_donor data to include organism_type value.")
    client = bigquery.Client()
    target_file = "anvil_donor.json"
    destination_dir = "ingest_pipeline/output/transformed/anvil/{}/table_data".format(dataset_id)
    query = """SELECT donor_id, 
    (SELECT MAX(CASE WHEN REGEXP_CONTAINS(value, '(h37|h38|h39|hg16|hg17|hg18|hg19|hs37|hs38|b37)') THEN 'Homo sapiens' END) AS organism_type FROM `{project}.{dataset}.workspace_attributes` WHERE attribute = 'library:reference') AS organism_type,
    part_of_dataset_id, phenotypic_sex, reported_ethnicity, genetic_ancestry, source_datarepo_row_ids
    FROM `{project}.{dataset}.anvil_donor`""".format(project=bq_project, dataset=bq_dataset)
    try:
        df = client.query(query).result().to_dataframe()
        records_json = df.to_json(orient='records') 
        records_list = json.loads(records_json)
        records_cnt = len(records_list)
        with open(target_file, 'w') as outfile:
            for idx, val in enumerate(records_list):
                json.dump(val, outfile)
                if idx < (records_cnt - 1):
                    outfile.write('\n')
        !gsutil cp $target_file $ws_bucket/$destination_dir/ 2> stdout
        !rm $target_file
        logging.info("Successfully created new anvil_donor.json file.")
    except Exception as e:
        logging.error("Error creating new anvil_donor.json file. Exiting function. Error: {}".format(str(e)))
        return "Failure"

    # Ingest updated anvil_donor data
    logging.info("Ingesting updated anvil_donor data into TDR dataset.")
    source_full_file_path = "{}/{}/{}".format(ws_bucket, destination_dir, "anvil_donor.json")
    ingest_request = {
        "table": "anvil_donor",
        "profile_id": "e0e03e48-5b96-45ec-baa4-8cc1ebf74c61",
        "ignore_unknown_values": True,
        "resolve_existing_files": True,
        "updateStrategy": "replace",
        "format": "json",
        "load_tag": "Ingest for {}".format(dataset_id),
        "path": source_full_file_path
    }
    attempt_counter = 0
    while True:
        try:
            api_client = utils.refresh_tdr_api_client()
            datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
            ingest_request_result, job_id = utils.wait_for_tdr_job(datasets_api.ingest_dataset(id=dataset_id, ingest=ingest_request))
            logging.info("Ingest from file anvil_donor.json succeeded: {}".format(str(ingest_request_result)[0:1000]))
            break
        except Exception as e:
            logging.error("Error on Dataset Ingest: {}".format(str(e)))
            attempt_counter += 1
            if attempt_counter < 2:
                logging.info("Retrying Dataset Ingest (attempt #{})...".format(str(attempt_counter)))
                sleep(10)
                continue
            else:
                logging.error("Maximum number of retries exceeded. Exiting function.")
                return "Failure"

    # Return success message if no failures recorded
    logging.info("Function completed successfully.")
    return "Success"

# Loop through datasets and process supplementary_file_flag
dataset_id_list = [
'd74b26d5-24bb-4696-84c3-bcd1f5f90b08',
'77ae9412-cd65-4bfb-9afb-29b77280ac33',
'9613ea77-4dd4-457c-b8b8-8d5fcde8fe20',
'f3d925d8-3aeb-49d0-8157-a6fa4c1a5056',
'1fe3865a-429f-43a1-bd19-1ebf023f452a',
'f7b1556b-950d-4f75-b570-c84f3d760119',
'f9b0ceaf-2a8f-4cfd-8d36-2107f47393cf',
'ef573845-3b10-48e6-8201-9579fa26d0e1',
'f1e1ef01-d52d-423e-a65b-3a1d26c7ee9d',
'd00353de-f6f9-42d9-8a8f-f88b3d880dbf',
'3be57453-9325-4c2e-b73a-832139b61778',
'fcc60ac9-0d20-4a7c-97e4-e3c8d3aa8f76',
'dc5f85d8-333b-4b68-b160-ad9856233887',
'841970b7-bed0-4a75-a28a-a4cc59740a84',
'fb5d9952-ebe7-4ee6-ba00-819ed00f3593',
'1c6bef41-3cfa-46b2-b183-0a523e417457',
'93b2ac60-2208-4ef8-a1c2-68a623e45807',
'ce58654d-b7d3-466b-99ba-b203d527a543',
'6905d8d1-da77-4f7c-86e5-3af7db2b00b4',
'59dafb62-a76f-4901-a778-46357f4bcd98',
'2a81cd6f-aa6e-436b-b4ba-68d5f713fb07',
'efcdb584-7659-4780-9d6d-e6599fb0033c',
'6c31262c-0141-47af-8145-e758dc949ea2',
'024c790a-3370-49b8-bef2-aee12afbd45e',
'95803570-3920-4ebc-8fa7-cf7aec78e4ed',
'c3386f43-2c70-4c5a-a489-5439a96607c8',
'175cc5db-36b1-4de2-851b-85b9645cf685',
'26358fe2-40da-497e-9f37-dc55de6b5527',
'576982f4-1581-4a81-88df-02c80ace0357',
'a3ae33bb-8b3a-47e5-a2d1-a49c954776b3',
'd2dca87f-f823-49d3-be17-649453ee47f6',
'6892958e-4f7e-404b-8755-dbbfdbf04f26',
'0e65b131-fd14-4fce-908b-c5b89a71a9c1',
'655e6a61-5400-4d8a-95bc-1506e026b289',
'1f2d14d4-1bd8-46fc-9d35-1a415e5f326a',
'64fd39fc-b32e-4b0a-8f83-4bf11b197462',
'b7d122c4-7328-41bd-a41e-0c5a2ba6afb0',
'21eb0406-94af-4b14-83ad-b6da45c425e7',
'158ebecd-4596-4541-b832-a137232b7036',
'1ccb95c3-1901-428e-b7bb-34495f41f4d2',
'02ff1051-cd1d-4bbb-a005-21384cbff846',
'0144b0d3-a809-46df-8c67-7ce42bdd579a',
'35a1009d-93a2-49b1-a801-fe84d6b7a2f5',
'50132478-c9fb-4dc5-86cd-d5dfab909393',
'35064fc1-6c52-4005-8e99-cb0d6afd3f8c',
'5cf859f6-990c-4b04-8609-35d5c57920f0',
'62cfdce6-2d4d-415c-a11e-5ab60131c668',
'2d07dd45-a263-440d-a339-9ccbab93aba8',
'296f653a-91a8-4139-9bab-e6ae13afe99c',
'633dc1aa-084d-43bd-9b17-bc6e57f81d48',
'9320b3b5-3944-4bd2-913a-23b72bccd86c',
'86ab4d3b-86ce-422b-ae6f-1ec6968a874d',
'3037caeb-fa7a-4924-b399-7e4c7173b3b8',
'146b72bc-1dcc-4e3b-bcda-d3dd25418012',
'1d575e14-c3b1-4ead-a63b-a21c08c6a14d',
'7f612c5a-ee5b-4a61-aa7f-e733ef301f19',
'1d2f5472-ab6a-4a9b-ba53-520858cf79db',
'3f278de3-f201-4344-9639-d35cd7a62adb',
'bf9108b6-bebc-4b3b-8517-6a2cce5f7d89',
'bf519ea2-afe1-486a-9954-7362f10b6b60',
'3a9604d7-456a-453d-a46b-40408624a07e',
'ff8b1212-858a-4048-8f63-9464c922591a',
'7eeede5a-c86f-4577-9f3c-65ab618a6dee',
'0194eea9-d779-4957-8521-11717a378e66',
'0d82658c-44b3-4cea-a388-3353a96a31ef',
'b5d7c34a-c383-4fc7-aa4d-b6dc941cd41a',
'c423b18d-12f2-43e4-97f9-993e2943270e',
'416b8daa-9537-46db-ae7b-3f5ff5f01dc3',
'2a263db0-8c33-4171-840f-54bf4755a4b9',
'424eb3a6-0f20-4c39-81ee-45cd3b1ca526',
'12bbfa4c-c30a-4cf6-b79f-45354f842964',
'319c6927-6c3d-4886-a164-02c226473bf6',
'c814d754-cdc5-4b0d-8671-a39e85b2c473',
'1b05159b-6277-4345-9d59-f7bba5ea1d56',
'92299ff4-c0d0-4e94-b374-75d0038cbd68',
'5205f817-7de0-48b4-89fc-6398cf13bff7',
'ceee2791-0fdf-45fc-a4e8-8077916771aa',
'41812f8d-65b4-4bcc-a674-651661f3c70e',
'd306000b-88c1-4220-8d7e-933c0118a983',
'ab76b5ca-e464-4063-b949-853f61036370',
'84fad495-2756-472f-ad20-f91de6f67baf',
'732eaae3-b509-4a7a-8961-09d861e55253',
'61940344-e6c1-484e-ba10-131f43a9b13a',
'35e3bace-335c-42eb-bba5-9e273e11daea',
'672b617f-936e-440a-a735-80f94798aed1',
'4124010f-7308-4831-80d7-ea14343249ab',
'af6c6f09-f0d2-46fe-bda0-c6fa5901c4a3',
'c6f3bd64-ea67-488f-904f-f0bdf6320b5c',
'0d379e59-5b32-4aa1-9055-ac57e7a100f1',
'a963c15d-9c97-49e4-af95-cdee96333a76',
'76dd508c-aa80-4e54-9ac4-23b5e0545316',
'c5b1e333-7203-41ce-b8f7-3ef3a3bd721f',
'a9ad3a05-24fb-4e59-85b0-ee09e55a4492',
'3abfc362-7e73-4663-9dcf-07b78b9aa2d4',
'6b40557c-ddc3-4e7e-8a45-1761e7fcb8b5',
'bb65d291-a673-4e4d-8a37-ab1f7401a902',
'1a7f6728-5116-4f24-897a-59a7f322cfd2',
'4999a410-990e-484b-b4f3-d636f894a741',
'4cd9b25c-97d0-4f0c-ad56-1e6ef5da57a3',
'bad1fb5c-d263-48d7-8e4c-fa873a17d707',
'01eaf423-8cab-491a-b82e-6915dbc73594',
'652cafc5-0131-48ea-b44d-3c59f1003eb2',
'64f2dbe1-6f58-493d-ab6b-c93568d828f4',
'a9626803-72c2-4e23-968c-a090e3f22c5e',
'8eff5518-fffb-4dce-bcf8-7f016300ee59',
'5137255f-0c58-4ac7-9266-bda8ab0247c2',
'c7e590be-a9f1-41a3-9d42-88dab277ea4a',
'f461fca1-80b2-4980-83a8-e165d49acc18',
'37f0f1f9-83fb-49a1-9941-093c068c32d0',
'cfb3dad7-c6d9-47c0-81b0-2133d75f5c0d',
'0481a135-9db1-424f-9065-a83ebd7ec995',
'902596ce-714e-49b3-8271-f3dfece52309',
'36dccf81-6932-43ae-9864-53379832d878',
'6fd45fc8-5c06-44b2-99da-5101f76e3fdf',
'a6c529da-f660-4a56-bdd2-d561cdd0ccd0',
'70eef9ca-c374-4dc9-b0d1-3f46eb8541c2',
'44161b51-953d-4f6b-9448-5cba4a44a9d8',
'0b90b2ea-8ca3-406a-9f69-95eddf7699ef',
'bbcf8529-1a04-43fc-b6cf-cb161028159d',
'027964c6-a26c-4eb1-9816-e02c5735cfe1',
'02cee0cd-4f77-41bb-a55c-f68c71611a86',
'85646f4a-e424-4363-8033-1e7522e8f175',
'42d4f99a-65cf-4c33-b54b-9d11d1c152cc',
'9d1241e0-2b6c-405f-a178-4d77c8971455',
'4d3c7031-244e-4d3c-a3e5-cf43fbbcab72',
'cc19d19e-6f7e-41b8-87a6-77f41d53e650',
'631deea0-2821-4d14-ad02-dc0ce4864924',
'f5f29e4b-68f7-443c-b290-0827d4167fd5',
'110e3635-1514-4e6f-809e-30eedde6d96e',
'6d07a2f8-c7de-4c90-b703-b258b80ab249',
'2cc2a071-b0f2-469d-a40c-802415c01587',
'16031a34-f1ba-4bde-af43-1822f1516944',
'8cc59f51-b0df-4a5d-a3c5-83ee526ff1af',
'033f16fd-656c-47d9-af03-a6b6e325c258',
'f8d5b908-2356-4775-8792-8611776dc257',
'be28577e-e1e1-4957-a38a-448053806acc',
'75fb0984-2124-444f-881b-30a1a6f8b8f7',
'700303c2-fcef-48a5-9900-096bf34e2d83',
'2d434f2c-6aaa-46b2-ada9-de4b887e13d3',
'267cf516-dd33-4640-a71a-78bd8f5db9d8',
'b2e7f15b-65d5-4812-abfd-b2dbc6d18850',
'd7bcfc5d-e258-4bd6-a413-bb7a118e6bff',
'2591451a-a8b9-411e-be9a-88fbc6660cb1',
'df06ff22-6a2d-4934-aac9-c8368efbea1a',
'2ebb722f-a3df-4ea4-b72a-813e3db0bab5',
'69f8d7c2-2e14-48e6-b838-7881016313fb',
'e03eb011-05f9-4491-b779-0cc2aefabff1',
'332bb145-6ef1-40ef-932c-aec5bb6210d9',
'b9842819-5fd6-40c5-9668-aae1ea44a308',
'2ee67bea-920c-41bd-885c-8128706f50ec',
'f3c88c3c-8e1b-4af9-9467-0621404e314c',
'7c056125-3ed8-459c-b73e-edfa3f80cc27',
'4b341ba9-49a5-43a2-9b7e-cc96beb59946',
'3376a8b6-7ef6-4191-97ab-a547da0d330d',
'1817528a-4f88-4ed6-8965-9eae0220ab27',
'5488d7c1-5195-4ebc-b0f0-31033fa06dc9',
'ec97fa0f-e174-40fe-a6b8-ee240bdf4318',
'56f9888f-e623-4a1a-b2b4-46378a6cd6fe',
'679e7d90-0d5c-484e-a0d2-b03705a60e7f',
'dd6c6688-b73a-464c-86d9-3369fdf98268',
'b108dfd0-711d-4bc1-aab5-1b312226c8ad',
'9dc31133-c882-4f39-903a-a25f316bb560',
'dcdefb14-f6de-4c46-ac7e-842b273416bf',
'53ce7d12-facc-4412-a710-f535efb209a2',
'90ba1853-f845-4502-ba36-b75b9e571bc5',
'7e37614e-add0-44ce-bd43-0acc0156cf5c',
'd4bb7169-5a7d-4090-ba62-12ea799c3ade',
'd61578a5-3f0c-4e67-8f5c-187be0714da5',
'724f393a-b3cc-47b3-8e08-37ebdbd06156',
'83010005-4891-4fc8-bc97-d9789f1bca2d',
'0a7d1d50-a6ea-438f-a2e2-a87a980e7a19',
'ed1215f1-787c-40f5-9d77-4b5bc2dfbb84',
'12e54f96-00d9-4a38-921e-e0d42610b2c1',
'0c6bc810-5ae0-4926-a56b-2bc2fe7dbe6d',
'dfb14a1f-38b2-4668-b98f-59b5b5b53ca3',
'bff73887-0497-4036-8131-1a43294aa832',
'43f25ff2-3c82-4c21-a8e5-50fcdcf7fe31',
'd049d487-1a69-4358-8dad-0e6fa6c06fdd',
'e922a496-e686-4fa1-911d-2159ceb0f09f',
'55d5643e-7b77-4beb-bcab-21bad45c0908',
'152c839c-45c9-4bbe-be26-3ba1869c43b7',
'6013a69f-26ee-4d1d-98d9-f52502501b27',
'02661394-2886-4ef7-aff1-d53225c82025',
'84133066-68cd-41fa-819b-d74a3ac85862',
'8de6dae2-55ff-4287-9b75-5b2a950c1f44',
'7511d432-ba87-4d5d-bf82-e74a396ae995',
'd3ed2595-b8be-40c8-b7b6-10a4997b9d2e',
'abbc650e-6b05-4ac9-926f-1931c122538a',
'61803dc8-f649-43e5-ab15-d351f2cef629',
'ce015292-898a-4c80-9249-abb671949406',
'abe58d43-e1c7-4953-aa41-4d3b6f6cca44',
'0d45c069-e0da-4d32-81e8-185a6ed96668',
'4809f175-afa3-4049-9884-c3991c547a09',
'0610d542-afc2-45b7-96ab-6c76e4a3c04c',
'29e73257-b5c4-43f0-9793-772f13d820b6',
'a1692d1f-a4a5-42e2-b944-48b07c72ee5d',
'939ff426-0846-4d81-8631-a7749a8e9b62',
'395da421-e6e8-4a26-ac93-eb7050a7cb1f',
'615f6246-1c39-4e44-a9d4-c7133a2ae62d',
'21384132-1697-4e9b-b863-a6492d13285d',
'dea7d0d6-e27a-4447-b06f-1136c6bab6e3',
'608d793e-a78b-4872-a50c-21a9eaa60ec3',
'af867604-d801-41cc-9949-017eb30a0cbf',
'a55f84ce-11c6-45e2-a3ae-dd049df56ac6',
'722e332c-fb1a-45fe-80c7-cc670f025b7f',
'1d140c76-a06b-42a0-bae8-b9e169ebe394',
'2a1cf830-4076-4f6b-8b98-820d8116d4b3',
'3615e063-f24b-47f7-87cb-430e8aca8d0c',
'e642bca0-52fb-4ab3-ab3a-acaab83deda7',
'9ecc231f-e3d3-4417-a98a-c4db4c638161',
'c911503c-f010-4c17-ac57-1d82e954bdc7',
'bf6f1d78-6a0d-4afb-aea6-17a3c34340db',
'23ddd300-e1e2-4aef-9839-10444a76bf27',
'c37b388c-7107-43d6-bee6-4e82b40ed271',
'f22bd762-5c45-453e-bf22-b174514abb84',
'7ea006d9-1e19-4678-b2e6-d4a1ea327f74',
'425412ba-894a-4824-acb8-bf18fe4576e0',
'd56ae233-d6d2-483c-917e-1de0fe1cfeb7',
'719f7581-21db-4aec-8c46-4a5811832710',
'8cc6f628-804b-4532-9954-10682882c9fc',
'15be288e-53e1-41cb-8d20-8ea87efb9258',
'0ee62643-b064-42f8-9b09-5d10eacd70a3',
'9ee2a552-89f8-4a48-9c94-9fa26ebb7483',
'ae4c80c8-a946-49cb-b376-81b4749f3221',
'dd2cb8fc-42a6-482f-898e-ef6125feccb8',
'2f47988f-d0fa-4019-8ea8-6e05fad0ef30',
'572fa0ee-3621-4350-a4ea-e2d2d040d053',
'29b16d47-bac7-4439-9ae2-c1866de36896',
'187fa022-6f24-4d7c-8570-f62786ea7610',
'339aa506-20e8-4dfe-9123-0e47aed7d176',
'7c111e99-8771-4c48-87ac-95f0f549a371',
'd3dc7286-d5b2-4a13-acbb-aa9d028d2b73',
'cf4b53fe-ab38-40bc-85d3-3a81b1eb5373',
'df99bd96-0b94-4796-a857-79dcfb1a794a',
'a34d06a0-629c-4641-8ff6-94aa21efb059',
'516ceb43-1378-4c02-88fc-a1d2a2258d59',
'8991fc1f-d056-47be-80e9-dca3cb94ea25',
'c383a955-ec11-40cd-908a-10c855120c4a',
'c7e9afbe-47be-4d31-bc5c-2b989bd3aa17',
'd4ba6130-acf9-4b58-9500-e4293eaa1d25',
'30f7621c-bff8-434a-90df-06f6e56e6031',
'10e173bd-e597-4ba2-b3cf-c959a38269f0',
'1fef2a90-dcdb-4501-a259-e7f81e4b2726',
'4a597ecd-3c6f-4a77-b927-cbc0e586a3e3',
'847b8433-c837-4c9c-ab30-704ca22cfeae',
'b164c54b-5bd7-4347-addf-0fd8b56f3254',
'9cb6aa04-01dc-4a9c-aa70-bfb7549d3db9',
'db8e3965-c129-431c-957e-6ddc0fe8c5d3',
'1481ed22-df3c-49b7-bfb8-8e814858c386',
'd61a2790-8ae8-4e2f-9fe9-0ba1a0e1dbf0',
'75949c06-891a-4dd5-94c8-82afbe47df03',
'13d5cde2-9044-4d32-ad45-2aeee48c79d0',
'e1b83751-576c-424a-816e-973bc91a5fdd',
'916433b8-455b-4bd2-85c0-fdfafc586629',
'921a3dbb-005a-457d-b57c-474d05553f21',
'6aaacba1-6b6c-48ab-9892-9c7436f27750',
'dc6e1ff6-1946-476b-9669-18583a439b20',
'10f34411-dbb0-4121-bc9b-10965f916f1b',
'c3747615-3e00-48e5-8705-9bb8a10a0c79',
'baa29e87-5319-4959-9810-58e4de8e9a54',
'ea20951a-8ae2-4a78-bd74-3ad9cba4f859',
'6139d218-d1fc-4e24-a4bc-080af788499f',
'197f2dab-1ea8-4b2c-b77a-1f690ca06151',
'6422e50d-5ba4-4a80-8409-4492216947ac',
'daf74a15-11b8-4fcc-82ee-7f013369ef35',
'f85ea65e-1943-4bd6-a541-71c5d8465ca9',
'd813b4e7-59f7-42a7-a52b-9358ffe0ceb3',
'2e60b5be-62c1-407a-aed9-c280109883e5',
'577f36fe-8154-4c82-ac87-b2a64cb68f35',
'2cbe079d-e7ab-47d8-836e-454a71440297',
'a3becdde-018b-46f0-adea-d587076eef4a',
'd0ce8b95-9c3b-4f9e-8ce0-169fd89a8b20',
'b252e3ac-4a8c-48e0-9999-5ee0c9a5842d',
'933c4de4-d48a-40c2-8fd6-c0b3e791a548',
'eec571be-e012-4ff8-a0e4-6fdf1841262e',
'f4a1cfa9-9b15-4ce8-afb0-75825acdac2f',
'5203f051-7e84-4969-b4ce-eda56a859793',
'c56f0a76-2b91-4860-8dff-63c9504bb0e2',
'73f7d2b4-86ec-4f7e-a1f9-37c7b023e3bf',
'151e70bd-c9a6-44f3-9c5c-43a0a79c0856',
'111b2e92-e69f-4fa0-b451-904c4a01331a',
'488a38ee-f996-482d-a562-a4474f5594de',
'165f825e-b506-4bbc-9476-774eb3a28cc8',
'84cf95d9-b0dd-4d97-ba09-ed832c07f4e0',
'269e867d-df41-43f0-8a3f-f62191b05a04',
'02cc5c7f-836b-45aa-b0c9-bac25196d614',
'0b0a52bb-a1a2-4638-9259-4447761c2da4',
'0eb42259-7b44-450f-a9d7-500b2ea7179c',
'039dd3d6-0cb5-4cd1-86b3-e9579c9b5218',
'e68d1d39-99df-4cd7-8053-1b298f03eabb',
'65793118-3c88-4185-9172-2354850e6056',
'36bdd59f-4f5b-43cd-8d34-a21ef87bbf30',
'a08dc7a6-f8ce-4205-95d2-83f614c2c32f',
'41cb9f29-4ba6-4690-821c-cb085e6b0f2f',
'35bfa7fa-0898-41d5-bfcf-8455b1e5921f',
'f934cabc-f19c-4515-82f3-a941bb400eca',
'140797da-dc94-4fc2-8b0b-f2e1dec7bd43',
'7cf0d3d0-f79b-4bfe-bfc8-e4e6c33dd4c3',
'c5d967fd-09ce-4b02-97dd-ac3abf6f79fa',
'017445d7-d56e-4e2e-b480-b4879b51e944',
'cb8ebcd0-bb5e-4a6d-bfef-5c651a1a9f6e',
'db266afc-2f75-4b03-a3b8-c69e0ce6f713',
'53d31bb7-b8fb-4622-b0c3-7ebfe81730eb',
'797b2563-5d56-4f5c-bdaf-3bfd11e8f5b3',
'a52c04ee-cfef-46bb-9b40-6a9b292e1a7b',
'85baa8f8-619c-4165-9d3e-53220f645814',
'28e73469-12d4-493b-bf6f-83359c1f69c5',
'aca4b572-df5c-4a75-85fa-1c2af15ca815',
'ec68371b-1d3a-4404-9900-d625cc5ca87a',
'8b8185d3-ba5c-4832-af23-3ff8ca6ed016',
'b60876c5-d825-4303-befb-ffff55b92aba',
'c9a68068-50e1-49d5-b061-54029a1fcdd1',
'329572b2-8826-419a-a948-66a2d7c0aa38',
'3fb2d04a-d18b-4bdc-9372-99b992f2ae42',
'c2fd0797-ca41-49a1-b485-a4bedac00613',
'51daecbd-37fa-4a58-8625-b6fad65acf27',
'5afc14bf-d7ca-4a62-b7aa-5104fa846888',
'af31b19a-9ea7-44e5-bc09-c9cace189b07',
'c93da7f4-227d-4652-9bba-1ae7c0e75af5',
'92e2f34d-4009-4260-a265-cd50331cdb73',
'69178fa1-87d4-4ecc-bc0e-7347c3678635',
'6d7bd12b-9198-48e4-af11-837eb3e32840',
'01e19c34-245a-40c1-ba4b-28b755996f60',
'19317283-9265-4dad-9901-62b901c2b512',
'cc107de7-d623-464a-a875-c8b7ae5fb09d',
'3c2c39a9-4cc2-4f7c-89e0-054a871e2c4e',
'6f49717f-8f57-42d0-8548-316ecc292415',
'7e693091-8ae4-4c40-8e66-c3b39f01b90e',
'60f96582-79ad-4461-9f9a-53c1bc3d17b6',
'9f9fc99a-b867-49a9-a3dc-8a39efbd5fa2',
'cba804c9-0bdd-4219-a53e-98c8db6334a0',
'5c6a1c4f-ccd3-48a8-ac00-e18e5ecaa0bb',
'12559f18-c36d-4d7f-80a9-cc0a8418e275',
'ddd3ffa8-8490-4ff9-9061-c884fa310f02',
'dc9f16b5-8fa2-462f-9652-67190dbeba54',
'b0979a88-8a59-4223-802d-6065ef57eeef',
'38fd20ce-affd-4791-9810-7f5a7fe876d0',
'28208cc6-50bf-4864-9a48-981632066640',
'f177843d-47fb-46ae-83be-73c92ee85081',
'dbf5d87b-4cab-44cd-a792-1d0218aad973',
'1c2fe11d-b020-4c54-8c71-1ea91623d626',
'3f172982-060d-4339-a09b-6994c2c9eb16',
'b2d13574-4e6c-4168-8117-fe7b202169bd',
'e6771964-50e9-482f-9d23-18c22cd89ab8',
'eede320a-ed63-41d8-960d-5405a26a194f',
'7f6358fa-c014-41d3-bd5f-de41848a08be',
'545c48b7-2630-4dab-87dd-4690988848f4',
'462c100f-08db-415c-b22f-e9654cee5518',
'865abeaf-6330-44d0-a989-dcc689df4531',
'f60cce71-083c-4407-8069-b76b1418aeff',
'80312f74-bd56-4938-96ba-e9bed95d1f3b',
'6b195cf7-1b5d-4678-b5af-23c265472db5',
'0b25d09e-b2d9-4452-9810-1d0ef777f9d6',
'e5c79b74-20d5-4b6f-8085-0bc788eed2ea',
'19e2c8ab-853a-4204-86c3-f591125fbf63',
'f3c89298-0dd2-40da-8627-3baea553b34a',
'173e56f7-b813-4c41-89ff-09a824e1407f',
'0114c84a-13ee-4c01-a164-d78457000fb1',
'0c18589c-6432-4a6c-90ce-985a47a66f39',
'3a72e4b8-afb4-4299-98ec-a9ba9606be06',
'bc6075ac-5cfe-4613-8601-36ceb614939e',
'ddc9e3fc-3f8b-4d2f-b4c6-4aade37b2fad',
'c215694a-f2df-442e-a2c8-04cfd01f2e03',
'44f83f20-d618-40b5-b2cb-3676b8fe3ad7',
'a5fe75bb-d28c-42fb-aaf8-92fa37b266d2',
'6545d602-e5b4-4dd1-8f6a-64e0a1952ddc',
'6765ce2d-ebc8-4367-8855-c0f8e62cb355',
'278a26cb-a710-4fff-928e-fc2e7084a75a',
'9fc492f3-8d13-47ae-93e9-812c0224f1aa',
'822d381e-cea0-45bb-8fa0-1b7194b4b64b',
'b7fb531e-25a4-427c-9679-b7bdc3d03535',
'74aee36b-f68b-43a4-8ae6-0d2797a1c4ad',
'ccf1d1c4-f9ad-4f12-8592-d61eb26cb4d4',
'9a32e23e-840d-4ba3-8cd9-392f48b8e9d2',
'5069fc2c-b957-4130-adca-6eabae943867',
'1939b7ae-fc6b-42a8-ad5f-dc51a1682a17',
'4e99b8e1-40b9-4fb2-90a0-d85e926ef31e',
'2cda53ba-b852-47e8-8f24-59ab8e9f1d1f',
'128332b6-5060-4ec4-b6a6-f53b54a810be',
'06f05f58-3c83-4f5c-bddd-bed7d2d1d147',
'9d796a02-e2aa-4c15-b8d6-1e90cd736681',
'433e3a09-661a-46a5-96f2-dbb07bdc87f3',
'34fd3b22-ac73-47d2-8849-5877158ec072',
'7ce3270e-b2f2-47f4-a288-639751b2f87f',
'fcb03f4f-e685-4803-aadb-0e8940ff4f37',
'41d12dc1-8718-4439-b409-26cc23573107',
'c2f0e7cf-ac07-48f7-b5f1-497ee6c134b2',
'c4c49fcd-0c20-4cff-841a-cb58f5689c5b',
'0d2c9df2-2c40-4683-a093-df731d033762',
'c5c0893f-b254-4038-8d08-b28ef5a26b5d',
'63b229b5-e7c8-4fd3-bbc8-ecf344da70d4',
'737d39b8-2f99-4eac-bcda-a03996e08939',
'31e61d00-61cc-46f2-a793-8ea8dfbb0832',
'254ffffc-2bd6-4b2e-905b-a8c54c348cd0',
'd6291444-8c3c-470c-b28c-7cf1d5c7aad8',
'ae34e63e-13af-48b8-8b72-8137289091b3',
'3fd2204c-8654-4af7-832f-c186447262e0',
'd48db47e-acba-4377-b031-f6dfc21f3658',
'575dc7da-58ed-407d-9e88-7b586f28bf65',
'fa278604-7d85-4491-a30d-15c7821f8b00',
'b5c0bf91-9d20-41a2-9dd2-87d0ef0310f9',
'32bc49c6-7583-4613-a72f-5edb12b2a808',
'3eb8ea77-4605-4bb7-90f9-671953abe4a2',
'2b08cb76-061d-44c6-a00f-b43a5421df5e',
'ab7e390a-adc5-4f9e-b317-a216a2904c93',
'd1e6d0e4-d49e-4a16-93c6-7956b2c03414',
'713f8676-8034-4827-bccc-cd6d95b1a4c4',
'b00883d8-9251-435d-aefc-8a703d96d2fb',
'eefbea02-0d65-441e-b455-35aa21d25ba3',
'ff7e3be2-c0ac-4d97-85da-6229bf7585ac',
'7ac92a42-e112-49c3-a8f5-8ad2c7ef5578',
'0701aae2-8661-4eec-84e0-7c8be1c89a18',
'7efb1905-34b4-4f1c-a8a6-8e64b3640a68',
'11a2b088-8c1c-47d2-9c1e-455d457d2f05',
'74608bd9-39e4-4f48-9b7c-1cd9d3c599c9',
'7baf8e8c-de11-452d-b2e1-aad7c08cc18e',
'23b0219d-0820-4017-b942-bda8578e90e2',
'a5f631ea-2b4b-43f2-9ea0-e31f2b11fa27',
'8523489a-f57c-4993-81e4-1ed86a5c092d',
'9cb5ce25-38e8-4628-9ddf-d6aedf5efe0f',
'a647528d-925e-4c02-8825-ff54720c6ee4',
'2c6f63b2-439e-499f-b687-b3fdd88a492e',
'68a916af-2e0c-41bd-8535-c7eacbc2d1b7',
'0e7f31a0-c712-4ebf-ab3a-64c37f43e52a',
'c8b1d323-f352-482e-bf17-82075c23dcee',
'd30f51c7-d642-4e7d-a168-967b9520a80a',
'8d89608c-0d61-4d71-a2e3-9fbc6cda69bf',
'71219f56-551f-4ad4-9a38-cc4aaf8a1e9a',
'48dd6010-77dc-465b-a27c-695e29b57a5e',
'582f5f8d-b96f-490e-b417-ba824baeb06c',
'7e825ee6-7c03-43cc-b0a4-0d9203a30bd9',
'2843292e-e494-4642-90e0-57e5c153f12c',
'4ecbb7c8-0246-47f8-9654-4caca1d52565',
'7e3ea1bd-95ba-4cad-90c8-3eec95be9cc8',
'bbba696b-d023-4bb1-a213-c8bee31e8bae',
'470eee0f-2053-4d9b-9f5e-ca9661a6cc16',
'5c1dc76d-b703-445c-9b38-cc2d00b9ab16',
'74ede771-6781-4980-bfb9-5d853b7cdd6f',
'6c47e282-5d5e-445c-b6bd-c0024946fbe0',
'ff8ffbcf-c932-48c7-8d5e-d995d5680e21',
'9d74b4f0-b2d4-46aa-867a-52fb6102bfdf',
'e34f15f7-c225-4314-a638-90504bb0aa0d',
'c1d222ab-bc0e-4e13-8379-0ee5be9e140e',
'582187a5-ad63-4759-9162-55fa6337eb07',
'd1e8d19a-970d-4ede-b5bc-9cab7237adec',
'9f152896-ebf1-4756-b678-bdf739a92256',
'478aa270-fbd4-4a45-8f63-221b4066168e',
'0b06619d-39d9-4437-8c42-2e415faa634c',
'e9c7ad29-2213-4648-9164-33a07bd42cdb',
'9e3fb02d-dcf6-486f-a42d-89446a852057',
'15ae6390-6f6d-4fd8-9a51-ecf988676c4d',
'3a3100bb-369e-47c1-a77c-2cacb7cf020d',
'2c11b505-17c8-402e-8422-0239accb449d',
'e25a8172-1e34-442c-a45d-583027a2d734',
'a6d7e030-e6c8-4c62-8cb5-165ef54987c4',
'128dce74-fa37-4f2f-8a80-d542edd81a11',
'e5d3e605-67a5-4317-b535-f75432700279',
'7feeb2b1-1926-4968-b6dc-e0a1e4cf8d4a',
'352a503b-41eb-4a84-b257-68d70e55337e',
'7b29f648-5404-4877-b353-e535e7f32a6a',
'd01a4268-1bfe-4a2d-a2d4-e296162c406e',
'e4ccd185-2b0c-445d-9c57-0dc45c8f9d7e',
'15d41c35-943c-474b-afa6-e1c6d6e4be2b',
'809185c9-5b1e-418b-878c-0a12519ef029',
'c9dd3578-01db-4687-9807-4f71368941d1',
'5edcc3db-c676-412a-9506-600959bb81f2',
'17a93d6f-8814-431d-855d-efbfbcc8fcda',
'3068f6fa-eada-475c-9571-d38bf1752878',
'ec4d928c-4f52-4581-b8fe-92b6e241f0e1',
'0d9400f9-013e-4e53-aa80-c8cb8022d3b5',
'ac4ecb0d-d56d-4330-b5a9-72acd5fe79a1',
'f8df3502-aeac-47a0-94c8-80758d082829',
'12720dd2-3513-4e0f-8ca0-109cc489e17f',
'c078b6ca-a3f3-4746-8631-9bb00eb5d954',
'47577b01-d7a7-497d-b261-183dade9cd84'
]
results = []
for dataset_id in dataset_id_list:
    status = populate_organism_type(dataset_id) 
    results.append([dataset_id, status])
    results_df = pd.DataFrame(results, columns = ["dataset_id", "run_status"])
display(results_df)


02/23/2023 05:38:24 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = d74b26d5-24bb-4696-84c3-bcd1f5f90b08
02/23/2023 05:38:24 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 05:38:24 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 05:38:32 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 05:38:32 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: SejRm1xBSKCbiKSBqGA9gA
02/23/2023 05:38:52 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': 'd74b26d5-24bb-4696-84c3-bcd1f5f90b08', 'dataset': 'ANVIL_CMG_UWash_GRU_20221020', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/d74b26d5-24bb-4696-84c3-bcd1f5f90b08/table_data/anvil_donor.json', 'load_tag': 'Ingest for d74b26d5-24bb-4696-84c3-bcd1f5f90b08', 'row_count': 2109, 'bad_row_count': 0, 'load_result': None}
02/23/2023 05:38:52 

02/23/2023 05:42:47 PM - INFO: Function completed successfully.
02/23/2023 05:42:47 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = f1e1ef01-d52d-423e-a65b-3a1d26c7ee9d
02/23/2023 05:42:47 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 05:42:47 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 05:42:53 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 05:42:53 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: woub9MJnSbaONA65GXR98w
02/23/2023 05:43:24 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': 'f1e1ef01-d52d-423e-a65b-3a1d26c7ee9d', 'dataset': 'ANVIL_CMG_UWASH_DS_BDIS_20221020', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/f1e1ef01-d52d-423e-a65b-3a1d26c7ee9d/table_data/anvil_donor.json', 'load_tag': 'Ingest for f1e1ef01-d52d-423e-a65b-3a1d26c7ee9d', 'row_count':

02/23/2023 05:47:38 PM - INFO: Function completed successfully.
02/23/2023 05:47:38 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = 93b2ac60-2208-4ef8-a1c2-68a623e45807
02/23/2023 05:47:38 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 05:47:38 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 05:47:44 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 05:47:44 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: N_9hQai_T76Q-vRHRn6amQ
02/23/2023 05:48:14 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': '93b2ac60-2208-4ef8-a1c2-68a623e45807', 'dataset': 'ANVIL_CMG_YALE_DS_MC_20221026', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/93b2ac60-2208-4ef8-a1c2-68a623e45807/table_data/anvil_donor.json', 'load_tag': 'Ingest for 93b2ac60-2208-4ef8-a1c2-68a623e45807', 'row_count': 79

02/23/2023 05:52:19 PM - INFO: Function completed successfully.
02/23/2023 05:52:19 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = 95803570-3920-4ebc-8fa7-cf7aec78e4ed
02/23/2023 05:52:19 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 05:52:20 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 05:52:26 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 05:52:26 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: 4w1rhFwGTtis0k-80_bZMw
02/23/2023 05:52:57 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': '95803570-3920-4ebc-8fa7-cf7aec78e4ed', 'dataset': 'ANVIL_CCDG_NYGC_AI_Asthma_Gala2_WGS_20221024', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/95803570-3920-4ebc-8fa7-cf7aec78e4ed/table_data/anvil_donor.json', 'load_tag': 'Ingest for 95803570-3920-4ebc-8fa7-cf7aec78e4ed', 

02/23/2023 05:57:07 PM - INFO: Function completed successfully.
02/23/2023 05:57:07 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = 0e65b131-fd14-4fce-908b-c5b89a71a9c1
02/23/2023 05:57:07 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 05:57:07 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 05:57:13 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 05:57:13 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: uGbXzdqqT_y7XdGi1fyFVA
02/23/2023 05:57:43 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': '0e65b131-fd14-4fce-908b-c5b89a71a9c1', 'dataset': 'ANVIL_ccdg_nygc_np_autism_tasc_wgs_20221024', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/0e65b131-fd14-4fce-908b-c5b89a71a9c1/table_data/anvil_donor.json', 'load_tag': 'Ingest for 0e65b131-fd14-4fce-908b-c5b89a71a9c1', '

...

02/23/2023 09:38:28 PM - INFO: Function completed successfully.
02/23/2023 09:38:28 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = b5c0bf91-9d20-41a2-9dd2-87d0ef0310f9
02/23/2023 09:38:28 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 09:38:29 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 09:38:34 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 09:38:34 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: Nq8Ng_fvQH2syPh0NWz2Ow
02/23/2023 09:39:05 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': 'b5c0bf91-9d20-41a2-9dd2-87d0ef0310f9', 'dataset': 'ANVIL_CCDG_Broad_NP_Epilepsy_BELULB_DS_EP_NPU_GSA_MD_20230118', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/b5c0bf91-9d20-41a2-9dd2-87d0ef0310f9/table_data/anvil_donor.json', 'load_tag': 'Ingest for b5c0bf91-9d20-41a2-9dd

02/23/2023 09:43:13 PM - INFO: Function completed successfully.
02/23/2023 09:43:13 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = eefbea02-0d65-441e-b455-35aa21d25ba3
02/23/2023 09:43:13 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 09:43:14 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 09:43:24 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 09:43:24 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: CQJKF6O3QSa6TUz1I0PuXA
02/23/2023 09:43:54 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': 'eefbea02-0d65-441e-b455-35aa21d25ba3', 'dataset': 'ANVIL_CCDG_Broad_NP_Epilepsy_DEUUKB_HMB_NPU_MDS_GSA_MD_20221128', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/eefbea02-0d65-441e-b455-35aa21d25ba3/table_data/anvil_donor.json', 'load_tag': 'Ingest for eefbea02-0d65-441e-b

02/23/2023 09:48:32 PM - INFO: Function completed successfully.
02/23/2023 09:48:32 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = 23b0219d-0820-4017-b942-bda8578e90e2
02/23/2023 09:48:32 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 09:48:32 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 09:48:38 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 09:48:38 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: RANw_pkQQv6o77maPdIjog
02/23/2023 09:49:09 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': '23b0219d-0820-4017-b942-bda8578e90e2', 'dataset': 'ANVIL_CCDG_Broad_NP_Epilepsy_GBRUCL_DS_EARET_MDS_GSA_MD_20221129', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/23b0219d-0820-4017-b942-bda8578e90e2/table_data/anvil_donor.json', 'load_tag': 'Ingest for 23b0219d-0820-4017-

02/23/2023 09:53:16 PM - INFO: Function completed successfully.
02/23/2023 09:53:16 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = c8b1d323-f352-482e-bf17-82075c23dcee
02/23/2023 09:53:16 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 09:53:16 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 09:53:22 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 09:53:22 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: 81hDFWJSQ3u1-L3ewBBDcg
02/23/2023 09:53:52 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': 'c8b1d323-f352-482e-bf17-82075c23dcee', 'dataset': 'ANVIL_CCDG_Broad_NP_Epilepsy_ITAUBG_DS_EPI_NPU_MDS_GSA_MD_20221129', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/c8b1d323-f352-482e-bf17-82075c23dcee/table_data/anvil_donor.json', 'load_tag': 'Ingest for c8b1d323-f352-482

02/23/2023 09:58:11 PM - INFO: Function completed successfully.
02/23/2023 09:58:11 PM - INFO: Processing anvil_donor.organism_type for Dataset ID = 4ecbb7c8-0246-47f8-9654-4caca1d52565
02/23/2023 09:58:11 PM - INFO: Retrieving necessary information from TDR.
02/23/2023 09:58:11 PM - INFO: Re-processing existing anvil_donor data to include organism_type value.
02/23/2023 09:58:17 PM - INFO: Successfully created new anvil_donor.json file.
02/23/2023 09:58:17 PM - INFO: Ingesting updated anvil_donor data into TDR dataset.
TDR Job ID: 4Yxo-2_sRZidNpyEIZOIaQ
02/23/2023 09:58:47 PM - INFO: Ingest from file anvil_donor.json succeeded: {'dataset_id': '4ecbb7c8-0246-47f8-9654-4caca1d52565', 'dataset': 'ANVIL_CCDG_Broad_NP_Epilepsy_NZLUTO_EPIL_BC_ID_MDS_GSA_MD_20221130', 'table': 'anvil_donor', 'path': 'gs://fc-2a9eefc3-0302-427f-9ac3-82f078741c03/ingest_pipeline/output/transformed/anvil/4ecbb7c8-0246-47f8-9654-4caca1d52565/table_data/anvil_donor.json', 'load_tag': 'Ingest for 4ecbb7c8-0246-47f

## Script to examine organism_type population

In [3]:
def validate_organism_type(dataset_id):
    
    # Retrieve dataset information
    src_schema_dict = {}
    api_client = utils.refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    try:
        response = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA", "ACCESS_INFORMATION"]).to_dict()
        src_schema_dict["tables"] = response["schema"]["tables"]
        bq_project = response["access_information"]["big_query"]["project_id"]
        bq_dataset = response["access_information"]["big_query"]["dataset_name"]
    except Exception as e:
        return "Failure - Issue Retrieving Dataset Info"
    
    # Determine if field exists for dataset, continue if so, fail otherwise
    client = bigquery.Client()
    query = """SELECT COUNT(organism_type) AS populated_cnt
                FROM `{project}.{dataset}.anvil_donor`""".format(project=bq_project, dataset=bq_dataset)
    try:
        df = client.query(query).result().to_dataframe()
        if df["populated_cnt"].values[0] > 0:
            return "Success - Field Populated"
        else:
            return "Success - Field Not Populated"
    except Exception as e:
        return "Failure - BigQuery Error"

# Loop through datasets and validate is_supplementary field
dataset_id_list = [
'd74b26d5-24bb-4696-84c3-bcd1f5f90b08',
]
results = []
for dataset_id in dataset_id_list:
    status = validate_organism_type(dataset_id) 
    results.append([dataset_id, status])
    results_df = pd.DataFrame(results, columns = ["dataset_id", "validation_status"])
display(results_df)

,dataset_id,validation_status
0,d74b26d5-24bb-4696-84c3-bcd1f5f90b08,Success - Field Populated
